<a href="https://colab.research.google.com/github/kslote1/HW-SIAM/blob/main/Online_Hui_Walter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np

In [39]:
class OnlineEMHuiWalter:
    def __init__(self, K, J, priors_init, se_init, sp_init, step_size=0.1):
        self.K = K
        self.J = J
        self.priors = np.array(priors_init)
        self.se = np.array(se_init)
        self.sp = np.array(sp_init)
        self.step_size = step_size

    def _update(self, data):
        K, J = self.K, self.J
        priors, se, sp = self.priors, self.se, self.sp
        #print(priors, se, sp)
        #input()
        step_size = self.step_size

        # Expectation step: calculate responsibilities
        denom = np.zeros_like(data)

        for k in range(K):
            for j in range(J):
                denom[:, j] += priors[k] * ((data[:, j] == 1) * se[j] + (data[:, j] == 0) * (1 - sp[j]))**k
        resp = np.zeros((data.shape[0], K))
        for k in range(K):
            for j in range(J):
                resp[:, k] += priors[k] * ((data[:, j] == 1) * se[j] + (data[:, j] == 0) * (1 - sp[j]))#**k
            resp[:, k] /= denom[:, k]

        # Maximization step: update the parameters
        priors_new = priors + step_size * (resp.mean(axis=0) - priors)
        se_new = se.copy()
        sp_new = sp.copy()
        epsilon = 1e-8
        for j in range(J):
            for k in range(K):
                se_new[j] += step_size * ((resp[:, k] * (data[:, j] == 1)).sum() / (resp[:, k].sum() + epsilon) - se[j])
                sp_new[j] += step_size * ((resp[:, k] * (data[:, j] == 0)).sum() / (resp[:, k].sum() + epsilon) - (1 - sp[j]))


        # Update the parameters
        self.priors = priors_new
        self.se = se_new
        self.sp = sp_new


    def update(self, data):
        K, J = self.K, self.J
        priors, se, sp = self.priors, self.se, self.sp
        step_size = self.step_size

        # Expectation step: calculate responsibilities
        denom = np.zeros_like(data)
        for k in range(K):
            for j in range(J):
                denom[:, j] += priors[k] * ((data[:, j] == 1) * se[j] + (data[:, j] == 0) * (1 - sp[j])) ** k

        resp = np.zeros((data.shape[0], K))
        for k in range(K):
            for j in range(J):
                resp[:, k] += priors[k] * ((data[:, j] == 1) * se[j] + (data[:, j] == 0) * (1 - sp[j])) ** k
            resp[:, k] /= denom[:, k]

        # Maximization step: update the parameters
        priors_new = priors + step_size * (resp.mean(axis=0) - priors)
        se_new = se.copy()
        sp_new = sp.copy()
        epsilon = 1e-8
        for j in range(J):
            se_denom = np.zeros(K)
            sp_denom = np.zeros(K)
            for k in range(K):
                se_denom[k] = (resp[:, k] * (data[:, j] == 1)).sum() + epsilon
                sp_denom[k] = (resp[:, k] * (data[:, j] == 0)).sum() + epsilon

            for k in range(K):
                se_new[j] += step_size * ((resp[:, k] * (data[:, j] == 1)).sum() / se_denom[k] - se[j])
                sp_new[j] += step_size * ((resp[:, k] * (data[:, j] == 0)).sum() / sp_denom[k] - (1 - sp[j]))

        # Normalize updates for se_new and sp_new
        se_new = np.clip(se_new, 0, 1)
        sp_new = np.clip(sp_new, 0, 1)

        # Update the parameters
        self.priors = priors_new
        self.se = se_new
        self.sp = sp_new

    def get_params(self):
        return self.priors, self.se, self.sp

class LogOnlineEMHuiWalter:
    def __init__(self, K, J, priors_init, se_init, sp_init, step_size=0.1):
        self.K = K
        self.J = J
        self.log_priors = np.log(np.array(priors_init))
        self.log_se = np.log(np.array(se_init))
        self.log_sp = np.log(np.array(sp_init))
        self.log_one_minus_sp = np.log(1 - np.array(sp_init))
        self.step_size = step_size

    def update(self, data):
        K, J = self.K, self.J
        log_priors, log_se, log_sp, log_one_minus_sp = self.log_priors, self.log_se, self.log_sp, self.log_one_minus_sp
        print(log_priors, log_se, log_sp, log_one_minus_sp)
        input()
        step_size = self.step_size

        # Expectation step: calculate responsibilities in log space
        log_denom = np.full_like(data, -np.inf)  # Use -inf for log(0)

        for k in range(K):
            for j in range(J):
                term = log_priors[k] + k * ((data[:, j] == 1) * log_se[j] + (data[:, j] == 0) * log_one_minus_sp[j])
                log_denom[:, j] = np.logaddexp(log_denom[:, j], term)  # log-sum-exp trick

        log_resp = np.full((data.shape[0], K), -np.inf)
        for k in range(K):
            for j in range(J):
                term = log_priors[k] + k * ((data[:, j] == 1) * log_se[j] + (data[:, j] == 0) * log_one_minus_sp[j])
                log_resp[:, k] = np.logaddexp(log_resp[:, k], term)
            log_resp[:, k] -= log_denom[:, j]

        # Maximization step: update the parameters in log space
        priors_new = np.exp(log_priors) + step_size * (np.mean(np.exp(log_resp), axis=0) - np.exp(log_priors))
        se_new = np.exp(log_se).copy()
        sp_new = np.exp(log_sp).copy()
        epsilon = 1e-8

        for j in range(J):
            for k in range(K):
                # Calculations for se_new and sp_new must be done in the original space since they involve non-logarithmic operations
                se_new[j] += step_size * ((np.exp(log_resp[:, k]) * (data[:, j] == 1)).sum() / (np.exp(log_resp[:, k]).sum() + epsilon) - se_new[j])
                sp_new[j] += step_size * ((np.exp(log_resp[:, k]) * (data[:, j] == 0)).sum() / (np.exp(log_resp[:, k]).sum() + epsilon) - (1 - sp_new[j]))

        # Update the parameters in log space
        self.log_priors = np.log(priors_new)
        self.log_se = np.log(se_new)
        self.log_sp = np.log(sp_new)
        self.log_one_minus_sp = np.log(1 - sp_new)

    def get_params(self):
        return np.exp(self.log_priors), np.exp(self.log_se), np.exp(self.log_sp)


In [50]:
class OnlineEMHuiWalter:
    def __init__(self, K, J, priors_init, se_init, sp_init, step_size=0.1):
        self.K = K
        self.J = J
        self.priors = np.array(priors_init)
        self.se = np.array(se_init)
        self.sp = np.array(sp_init)
        self.step_size = step_size

    def _update(self, data):
        K, J = self.K, self.J
        priors, se, sp = self.priors, self.se, self.sp
        step_size = self.step_size

        # Expectation step: calculate responsibilities
        resp = np.zeros((data.shape[0], K))
        for k in range(K):
            likelihood_k = np.ones(data.shape[0])
            for j in range(J):
                likelihood_k *= (data[:, j] * se[j] + (1 - data[:, j]) * (1 - sp[j])) ** k
            resp[:, k] = priors[k] * likelihood_k

        resp_sum = resp.sum(axis=1, keepdims=True)
        resp /= resp_sum + 1e-8

        # Maximization step: update the parameters
        priors_new = np.clip(priors + step_size * (resp.mean(axis=0) - priors), 0, 1)

        se_new = se.copy()
        sp_new = sp.copy()
        for j in range(J):
            for k in range(K):
                se_new[j] = np.clip(se[j] + step_size * ((resp[:, k] * data[:, j]).sum() / (resp[:, k].sum() + 1e-8) - se[j]), 0, 1)
                sp_new[j] = np.clip(sp[j] + step_size * ((resp[:, k] * (1 - data[:, j])).sum() / (resp[:, k].sum() + 1e-8) - (1 - sp[j])), 0, 1)

        # Update the parameters
        self.priors = priors_new
        self.se = se_new
        self.sp = sp_new

    def update(self, data):
        K, J = self.K, self.J
        priors, se, sp = self.priors, self.se, self.sp
        step_size = self.step_size
        epsilon = 1e-10

        # Calculate negative log-likelihood and its gradient for the data point
        neg_log_likelihood = 0
        grad_priors = np.zeros(K)
        grad_se = np.zeros(J)
        grad_sp = np.zeros(J)

        # Loop over each class and test
        for k in range(K):
            likelihood_k = priors[k]
            for j in range(J):
                # Iterating over each element in data[j]
                for i in range(data.shape[0]):
                    p_success = (data[i, j] * se[j] + (1 - data[i, j]) * (1 - sp[j])) ** k
                    likelihood_k *= p_success
                    # Compute gradients
                    if p_success > 0:
                        grad_priors[k] += (data[i, j] - p_success) / p_success
                        grad_se[j] += k * ((data[i, j] - se[j]) / p_success)
                        grad_sp[j] += k * (((1 - data[i, j]) - (1 - sp[j])) / p_success)

            neg_log_likelihood -= np.log(likelihood_k + epsilon)
        # Update parameters using gradients
        priors_new = np.clip(priors - step_size * grad_priors, 0, 1)
        se_new = np.clip(se - step_size * grad_se, 0, 1)
        sp_new = np.clip(sp - step_size * grad_sp, 0, 1)

        # Update the parameters
        self.priors = priors_new
        self.se = se_new
        self.sp = sp_new


    def get_params(self):
        return self.priors, self.se, self.sp


In [ ]:
class OnlineEMHuiWalter:
    def __init__(self, K, J, priors_init, se_init, sp_init, step_size=0.1):
        self.K = K
        self.J = J
        self.priors = np.array(priors_init)
        self.se = np.array(se_init)
        self.sp = np.array(sp_init)
        self.step_size = step_size

    def update(self, data):
        K, J = self.K, self.J
        priors, se, sp = self.priors, self.se, self.sp
        step_size = self.step_size
        epsilon = 1e-10

        # Calculate negative log-likelihood and its gradient for the data point
        neg_log_likelihood = 0
        grad_priors = np.zeros(K)
        grad_se = np.zeros(J)
        grad_sp = np.zeros(J)

        # Loop over each class and test
        for k in range(K):
            likelihood_k = priors[k]
            for j in range(J):
                # Iterating over each element in data[j]
                for i in range(data.shape[0]):
                    p_success = (data[i, j] * se[j] + (1 - data[i, j]) * (1 - sp[j])) ** k
                    likelihood_k *= p_success
                    # Compute gradients
                    if p_success > 0:
                        grad_priors[k] += (data[i, j] - p_success) / p_success
                        grad_se[j] += k * ((data[i, j] - se[j]) / p_success)
                        grad_sp[j] += k * (((1 - data[i, j]) - (1 - sp[j])) / p_success)

            neg_log_likelihood -= np.log(likelihood_k + epsilon)
        # Update parameters using gradients
        priors_new = np.clip(priors - step_size * grad_priors, 0, 1)
        se_new = np.clip(se - step_size * grad_se, 0, 1)
        sp_new = np.clip(sp - step_size * grad_sp, 0, 1)

        # Update the parameters
        self.priors = priors_new
        self.se = se_new
        self.sp = sp_new


    def get_params(self):
        return self.priors, self.se, self.sp

In [2]:
class OnlineEMHuiWalter:
    def __init__(self, K, J, priors_init, se_init, sp_init, step_size=0.1):
        self.K = K
        self.J = J
        self.priors = np.array(priors_init)
        self.se = np.array(se_init)
        self.sp = np.array(sp_init)
        self.step_size = step_size

    def update(self, data):
        K, J = self.K, self.J
        priors, se, sp = self.priors, self.se, self.sp
        step_size = self.step_size
        epsilon = 1e-10

        # Calculate negative log-likelihood and its gradient for the data point
        #neg_log_likelihood = 0
        grad_priors = np.zeros(K)
        grad_se = np.zeros(J)
        grad_sp = np.zeros(J)

        # Loop over each class and test
        for k in range(K):
            likelihood_k = priors[k]
            for j in range(J):
                # Iterating over each element in data[j]
                for i in range(data.shape[0]):
                    p_success = (data[i, j] * se[j] + (1 - data[i, j]) * (1 - sp[j])) ** k
                    likelihood_k *= p_success
                    # Compute gradients
                    if p_success > 0:
                        grad_priors[k] += (data[i, j] - p_success) / p_success
                        grad_se[j] += k * ((data[i, j] - se[j]) / p_success)
                        grad_sp[j] += k * (((1 - data[i, j]) * (1 - sp[j])) / p_success)

            #neg_log_likelihood -= np.log(likelihood_k + epsilon)
        # Update parameters using gradients
        priors_new = np.clip(priors - step_size * grad_priors, 0, 1)
        se_new = np.clip(se - step_size * grad_se, 0, 1)
        sp_new = np.clip(sp - step_size * grad_sp, 0, 1)

        # Update the parameters
        self.priors = priors_new
        self.se = se_new
        self.sp = sp_new


    def get_params(self):
        return self.priors, self.se, self.sp

In [5]:
def generate_data(n_samples, priors, se, sp):
    n_classes = len(priors)
    n_tests = len(se)

    data = np.zeros((n_samples, n_tests))
    true_classes = np.random.choice(n_classes, n_samples, p=priors)
    #print(true_classes)

    for i, c in enumerate(true_classes):
        for j in range(n_tests):
            if c == 1:
                data[i, j] = np.random.choice([0, 1], p=[1-se[j], se[j]])
            else:
                data[i, j] = np.random.choice([0, 1], p=[sp[j], 1-sp[j]])

    return data

# True parameters for generating synthetic data
true_priors = [0.6, 0.4]
true_se = [0.9, 0.7]
true_sp = [0.8, 0.6]

# Generate synthetic data
n_samples = 10000
data = generate_data(n_samples, true_priors, true_se, true_sp)

# Initialize the OnlineEMHuiWalter class
priors_init = [0.5, 0.5]
se_init = [0.8, 0.6]
sp_init = [0.7, 0.56]
step_size = 0.0001
em = OnlineEMHuiWalter(2, 2, priors_init, se_init, sp_init, step_size)

# Update the OnlineEMHuiWalter instance with the generated data
for i in range(n_samples):
    em.update(data[i].reshape(1, -1))
    #print(em.get_params())

# Get the estimated parameters
estimated_priors, estimated_se, estimated_sp = em.get_params()

# Print the true and estimated parameters
print("True priors:", true_priors)
print("Estimated priors:", estimated_priors)
print("True se:", true_se)
print("Estimated se:", estimated_se)
print("True sp:", true_sp)
print("Estimated sp:", estimated_sp)


True priors: [0.6, 0.4]
Estimated priors: [1. 1.]
True se: [0.9, 0.7]
Estimated se: [1. 1.]
True sp: [0.8, 0.6]
Estimated sp: [0.1788 0.0736]


In [ ]:
from scipy.optimize import minimize
import numpy as np


class OnlineEMHuiWalter:
    def __init__(self, num_classes, num_categories,
                 params, step_size=0.1):
        self.num_classes = num_classes
        self.num_categories = num_categories
        assert num_classes == 2
        assert num_categories == 2
        self.params = params

    def expectation_step(self,  data):
        # Unpack the parameters
        alpha1, alpha2, beta1, beta2, theta1, theta2 = self.params

        # Unpack the data
        #X1, X2, X3, X4, X5, X6, X7, X8 = data

        X1 = np.sum(data[:, 0] == 1)
        X2 = np.sum(data[:, 0] != data[:, 0])
        X3 = np.sum(data[:, 0] != data[:, 1])
        X4 = np.sum(data[:, 0] == 0)

        X5 = np.sum(data[:, 1] == 1)
        X6 = np.sum(data[:, 1] != data[:, 0])
        X7 = np.sum(data[:, 1] != data[:, 1])
        X8 = np.sum(data[:, 1] == 0)

        # Calculate total counts for each population
        tot1 = X1 + X2 + X3 + X4
        tot2 = X5 + X6 + X7 + X8

        # Calculate probabilities for each cell in the first population
        p1 = theta1 * (1 - beta1) * (1 - beta2) + (1 - theta1) * alpha1 * alpha2
        p2 = theta1 * (1 - beta1) * beta2 + (1 - theta1) * alpha1 * (1 - alpha2)
        p3 = theta1 * beta1 * (1 - beta2) + (1 - theta1) * (1 - alpha1) * alpha2
        p4 = theta1 * beta1 * beta2 + (1 - theta1) * (1 - alpha1) * (1 - alpha2)

        # Calculate probabilities for each cell in the second population
        p5 = theta2 * (1 - beta1) * (1 - beta2) + (1 - theta2) * alpha1 * alpha2
        p6 = theta2 * (1 - beta1) * beta2 + (1 - theta2) * alpha1 * (1 - alpha2)
        p7 = theta2 * beta1 * (1 - beta2) + (1 - theta2) * (1 - alpha1) * alpha2
        p8 = theta2 * beta1 * beta2 + (1 - theta2) * (1 - alpha1) * (1 - alpha2)

        # Expected counts for each cell
        expected_counts = [
            tot1 * p1, tot1 * p2, tot1 * p3, tot1 * p4,
            tot2 * p5, tot2 * p6, tot2 * p7, tot2 * p8
        ]

        return expected_counts


    def update(self, data):
        # Unpack the parameters
        alpha1, alpha2, beta1, beta2, theta1, theta2 = self.params
        # Unpack the parameters
        ex_alpha1, ex_alpha2, ex_beta1, ex_beta2, ex_theta1, ex_theta2 = self.expectation_step(data)

        # Update paramaters using expectation maximization


def generate_data(n_samples, priors, se, sp):
    n_classes = len(priors)
    n_tests = len(se)

    data = np.zeros((n_samples, n_tests))
    true_classes = np.random.choice(n_classes, n_samples, p=priors)
    #print(true_classes)

    for i, c in enumerate(true_classes):
        for j in range(n_tests):
            if c == 1:
                data[i, j] = np.random.choice([0, 1], p=[1-se[j], se[j]])
            else:
                data[i, j] = np.random.choice([0, 1], p=[sp[j], 1-sp[j]])

    return data

# True parameters for generating synthetic data
true_priors = [0.6, 0.4]
true_se = [0.9, 0.7]
true_sp = [0.8, 0.6]

# Generate synthetic data
n_samples = 10000
data = generate_data(n_samples, true_priors, true_se, true_sp)
# Initialize the OnlineEMHuiWalter class
params = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
step_size = 0.0001
em = OnlineEMHuiWalter(2, 2, params, step_size)
for i in range(n_samples):
    em.update(data[i].reshape(1, -1))

In [26]:
from scipy.optimize import minimize
import numpy as np


class OnlineEMHuiWalter:
    def __init__(self, num_classes, num_categories,
                 params, step_size=0.1):
        self.num_classes = num_classes
        self.num_categories = num_categories
        assert num_classes == 2
        assert num_categories == 2
        self.params = params
        self.step_size = step_size

    def expectation_step(self,  data):
        # Unpack the parameters
        alpha1, alpha2, beta1, beta2, theta1, theta2 = self.params

        # Unpack the data
        #X1, X2, X3, X4, X5, X6, X7, X8 = data

        X1 = np.sum(data[:, 0] == 1)
        X2 = np.sum(data[:, 0] != data[:, 0])
        X3 = np.sum(data[:, 0] != data[:, 1])
        X4 = np.sum(data[:, 0] == 0)

        X5 = np.sum(data[:, 1] == 1)
        X6 = np.sum(data[:, 1] != data[:, 0])
        X7 = np.sum(data[:, 1] != data[:, 1])
        X8 = np.sum(data[:, 1] == 0)

        # Calculate total counts for each population
        tot1 = X1 + X2 + X3 + X4
        tot2 = X5 + X6 + X7 + X8

        # Calculate probabilities for each cell in the first population
        p1 = theta1 * (1 - beta1) * (1 - beta2) + (1 - theta1) * alpha1 * alpha2
        p2 = theta1 * (1 - beta1) * beta2 + (1 - theta1) * alpha1 * (1 - alpha2)
        p3 = theta1 * beta1 * (1 - beta2) + (1 - theta1) * (1 - alpha1) * alpha2
        p4 = theta1 * beta1 * beta2 + (1 - theta1) * (1 - alpha1) * (1 - alpha2)

        # Calculate probabilities for each cell in the second population
        p5 = theta2 * (1 - beta1) * (1 - beta2) + (1 - theta2) * alpha1 * alpha2
        p6 = theta2 * (1 - beta1) * beta2 + (1 - theta2) * alpha1 * (1 - alpha2)
        p7 = theta2 * beta1 * (1 - beta2) + (1 - theta2) * (1 - alpha1) * alpha2
        p8 = theta2 * beta1 * beta2 + (1 - theta2) * (1 - alpha1) * (1 - alpha2)

        # Expected counts for each cell
        expected_counts = [
            tot1 * p1, tot1 * p2, tot1 * p3, tot1 * p4,
            tot2 * p5, tot2 * p6, tot2 * p7, tot2 * p8
        ]

        cell_probs = [
            p1, p2, p3, p4,
            p5, p6, p7, p8
        ]

        return expected_counts, cell_probs

    def calculate_probabilities(self, alpha1, alpha2, beta1, beta2, theta1, theta2):
        # Calculate probabilities for each cell in the first population
        p1 = theta1 * (1 - beta1) * (1 - beta2) + (1 - theta1) * alpha1 * alpha2
        p2 = theta1 * (1 - beta1) * beta2 + (1 - theta1) * alpha1 * (1 - alpha2)
        p3 = theta1 * beta1 * (1 - beta2) + (1 - theta1) * (1 - alpha1) * alpha2
        p4 = theta1 * beta1 * beta2 + (1 - theta1) * (1 - alpha1) * (1 - alpha2)

        # Calculate probabilities for each cell in the second population
        p5 = theta2 * (1 - beta1) * (1 - beta2) + (1 - theta2) * alpha1 * alpha2
        p6 = theta2 * (1 - beta1) * beta2 + (1 - theta2) * alpha1 * (1 - alpha2)
        p7 = theta2 * beta1 * (1 - beta2) + (1 - theta2) * (1 - alpha1) * alpha2
        p8 = theta2 * beta1 * beta2 + (1 - theta2) * (1 - alpha1) * (1 - alpha2)

        return p1, p2, p3, p4, p5, p6, p7, p8

    def negative_log_likelihood(self, data):
        # Unpack the parameters
        alpha1, alpha2, beta1, beta2, theta1, theta2 = self.params
        p1, p2, p3, p4, p5, p6, p7, p8 = self.calculate_probabilities(alpha1, alpha2, beta1, beta2, theta1, theta2)

        X1 = np.sum(data[:, 0] == 1)
        X2 = np.sum(data[:, 0] != data[:, 0])
        X3 = np.sum(data[:, 0] != data[:, 1])
        X4 = np.sum(data[:, 0] == 0)

        X5 = np.sum(data[:, 1] == 1)
        X6 = np.sum(data[:, 1] != data[:, 0])
        X7 = np.sum(data[:, 1] != data[:, 1])
        X8 = np.sum(data[:, 1] == 0)

        tot1 = X1 + X3 + X3 + X4
        tot2 =  X5 + X6 + X7 + X8

        eps = 1e-10
        L = (
            (p1 + eps)**(X1/tot1) *
            (p2 + eps)**(X2/tot1) *
            (p3 + eps)**(X3/tot1) *
            (p4 + eps)**(X4/tot1) *
            (p5 + eps)**(X5/tot2) *
            (p6 + eps)**(X6/tot2) *
            (p7 + eps)**(X7/tot2) *
            (p8 + eps)**(X8/tot2)
        )

        return -np.log(L) # Return the negative log-likelihood for minimization

    def inverse_calculate_probabilities(p1, p2, p3, p4, p5, p6, p7, p8):
        # Objective function to minimize
        def error_function(params):
            alpha1, alpha2, beta1, beta2, theta1, theta2 = params
            calc_p1, calc_p2, calc_p3, calc_p4, calc_p5, calc_p6, calc_p7, calc_p8 = self.calculate_probabilities(alpha1, alpha2, beta1, beta2, theta1, theta2)
            return ((calc_p1 - p1) ** 2 + (calc_p2 - p2) ** 2 +
                (calc_p3 - p3) ** 2 + (calc_p4 - p4) ** 2 +
                (calc_p5 - p5) ** 2 + (calc_p6 - p6) ** 2 +
                (calc_p7 - p7) ** 2 + (calc_p8 - p8) ** 2)

        # Initial guess
        initial_guess = self.params #[0.5, 0.5, 0.5, 0.5, 0.5, 0.5]

        # Perform minimization
        result = minimize(error_function, initial_guess, bounds=[(0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1)])

        if result.success:
            return result.x
        else:
            raise ValueError("Optimization failed: " + result.message)

    def _update(self, data):

         # Unpack the parameters
        alpha1, alpha2, beta1, beta2, theta1, theta2 = self.params

        responsibilities, call_probs = self.expectation_step(data)
        #print(responsibilities)

        p1, p2, p3, p4, p5, p6, p7, p8 = self.calculate_probabilities(alpha1, alpha2, beta1, beta2, theta1, theta2)

        # Update parameters with a step proportional to the responsibilities and step_size
        # Note: The actual update rules depend on your model's specifics
        alpha1 += self.step_size * (responsibilities[0] - responsibilities[1])
        alpha2 += self.step_size * (responsibilities[0] - responsibilities[2])
        beta1  += self.step_size * (responsibilities[1] - responsibilities[3])
        beta2  += self.step_size * (responsibilities[2] - responsibilities[3])
        theta1 += self.step_size * (sum(responsibilities[:4]) - 2)
        theta2 += self.step_size * (sum(responsibilities[4:]) - 2)

        # Ensure parameters remain within valid range
        self.params = [
            max(0, min(1, alpha1)), max(0, min(1, alpha2)),
            max(0, min(1, beta1)), max(0, min(1, beta2)),
            max(0, min(1, theta1)), max(0, min(1, theta2))
        ]

    def update(self, data):
        print(self.negative_log_likelihood(data))


def generate_data(n_samples, priors, se, sp):
    n_classes = len(priors)
    n_tests = len(se)

    data = np.zeros((n_samples, n_tests))
    true_classes = np.random.choice(n_classes, n_samples, p=priors)
    #print(true_classes)

    for i, c in enumerate(true_classes):
        for j in range(n_tests):
            if c == 1:
                data[i, j] = np.random.choice([0, 1], p=[1-se[j], se[j]])
            else:
                data[i, j] = np.random.choice([0, 1], p=[sp[j], 1-sp[j]])

    return data

# True parameters for generating synthetic data
true_priors = [0.6, 0.4]
true_se = [0.9, 0.7]
true_sp = [0.8, 0.6]

# Generate synthetic data
n_samples = 10000
data = generate_data(n_samples, true_priors, true_se, true_sp)
# Initialize the OnlineEMHuiWalter class
params = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
step_size = 0.0001
em = OnlineEMHuiWalter(2, 2, params, step_size)
for i in range(n_samples):
    em.update(data[i].reshape(1, -1))
    #print(em.params)

Streaming output truncated to the last 5000 lines.
2.772588721439781
2.3104906011998176
2.772588721439781
2.3104906011998176
2.3104906011998176
2.772588721439781
2.772588721439781
2.772588721439781
2.772588721439781
2.3104906011998176
2.772588721439781
2.772588721439781
2.3104906011998176
2.772588721439781
2.772588721439781
2.772588721439781
2.772588721439781
2.772588721439781
2.772588721439781
2.772588721439781
2.772588721439781
2.772588721439781
2.3104906011998176
2.772588721439781
2.772588721439781
2.3104906011998176
2.3104906011998176
2.3104906011998176
2.772588721439781
2.772588721439781
2.3104906011998176
2.772588721439781
2.3104906011998176
2.3104906011998176
2.772588721439781
2.772588721439781
2.772588721439781
2.772588721439781
2.772588721439781
2.772588721439781
2.772588721439781
2.772588721439781
2.772588721439781
2.3104906011998176
2.3104906011998176
2.772588721439781
2.3104906011998176
2.772588721439781
2.772588721439781
2.3104906011998176
2.3104906011998176
2.310490601199

In [7]:
data
disagreements = data[:, 0] != data[:, 1]
print(disagreements)

[ True False False ... False False False]


In [ ]:
from re import X
from scipy.optimize import minimize
import numpy as np


class OnlineEMHuiWalter:
    def __init__(self, num_classes, num_categories,
                 params, step_size=0.1):
        self.num_classes = num_classes
        self.num_categories = num_categories
        assert num_classes == 2
        assert num_categories == 2
        self.params = params
        self.step_size = step_size

    def expectation_step(self,  data):
        # Unpack the parameters
        alpha1, alpha2, beta1, beta2, theta1, theta2 = self.params

        # Unpack the data
        #X1, X2, X3, X4, X5, X6, X7, X8 = data

        X1 = np.sum(data[:, 0] == 1)
        X2 = np.sum(data[:, 0] != data[:, 0])
        X3 = np.sum(data[:, 0] != data[:, 1])
        X4 = np.sum(data[:, 0] == 0)

        X5 = np.sum(data[:, 1] == 1)
        X6 = np.sum(data[:, 1] != data[:, 0])
        X7 = np.sum(data[:, 1] != data[:, 1])
        X8 = np.sum(data[:, 1] == 0)

        # Calculate total counts for each population
        tot1 = X1 + X2 + X3 + X4
        tot2 = X5 + X6 + X7 + X8

        # Calculate probabilities for each cell in the first population
        p1 = theta1 * (1 - beta1) * (1 - beta2) + (1 - theta1) * alpha1 * alpha2
        p2 = theta1 * (1 - beta1) * beta2 + (1 - theta1) * alpha1 * (1 - alpha2)
        p3 = theta1 * beta1 * (1 - beta2) + (1 - theta1) * (1 - alpha1) * alpha2
        p4 = theta1 * beta1 * beta2 + (1 - theta1) * (1 - alpha1) * (1 - alpha2)

        # Calculate probabilities for each cell in the second population
        p5 = theta2 * (1 - beta1) * (1 - beta2) + (1 - theta2) * alpha1 * alpha2
        p6 = theta2 * (1 - beta1) * beta2 + (1 - theta2) * alpha1 * (1 - alpha2)
        p7 = theta2 * beta1 * (1 - beta2) + (1 - theta2) * (1 - alpha1) * alpha2
        p8 = theta2 * beta1 * beta2 + (1 - theta2) * (1 - alpha1) * (1 - alpha2)

        # Expected counts for each cell
        expected_counts = [
            tot1 * p1, tot1 * p2, tot1 * p3, tot1 * p4,
            tot2 * p5, tot2 * p6, tot2 * p7, tot2 * p8
        ]

        cell_probs = [
            p1, p2, p3, p4,
            p5, p6, p7, p8
        ]

        return expected_counts, cell_probs

    def calculate_probabilities(self, alpha1, alpha2, beta1, beta2, theta1, theta2):
        # Calculate probabilities for each cell in the first population
        p1 = theta1 * (1 - beta1) * (1 - beta2) + (1 - theta1) * alpha1 * alpha2
        p2 = theta1 * (1 - beta1) * beta2 + (1 - theta1) * alpha1 * (1 - alpha2)
        p3 = theta1 * beta1 * (1 - beta2) + (1 - theta1) * (1 - alpha1) * alpha2
        p4 = theta1 * beta1 * beta2 + (1 - theta1) * (1 - alpha1) * (1 - alpha2)

        # Calculate probabilities for each cell in the second population
        p5 = theta2 * (1 - beta1) * (1 - beta2) + (1 - theta2) * alpha1 * alpha2
        p6 = theta2 * (1 - beta1) * beta2 + (1 - theta2) * alpha1 * (1 - alpha2)
        p7 = theta2 * beta1 * (1 - beta2) + (1 - theta2) * (1 - alpha1) * alpha2
        p8 = theta2 * beta1 * beta2 + (1 - theta2) * (1 - alpha1) * (1 - alpha2)

        return p1, p2, p3, p4, p5, p6, p7, p8

    def negative_log_likelihood(self, data):
        # Unpack the parameters
        alpha1, alpha2, beta1, beta2, theta1, theta2 = self.params
        p1, p2, p3, p4, p5, p6, p7, p8 = self.calculate_probabilities(alpha1, alpha2, beta1, beta2, theta1, theta2)

        X1 = np.sum(data[:, 0] == 1)
        X2 = np.sum(data[:, 0] != data[:, 0])
        X3 = np.sum(data[:, 0] != data[:, 1])
        X4 = np.sum(data[:, 0] == 0)

        X5 = np.sum(data[:, 1] == 1)
        X6 = np.sum(data[:, 1] != data[:, 0])
        X7 = np.sum(data[:, 1] != data[:, 1])
        X8 = np.sum(data[:, 1] == 0)

        tot1 = X1 + X3 + X3 + X4
        tot2 =  X5 + X6 + X7 + X8

        eps = 1e-10
        L = (
            (p1 + eps)**(X1/tot1) *
            (p2 + eps)**(X2/tot1) *
            (p3 + eps)**(X3/tot1) *
            (p4 + eps)**(X4/tot1) *
            (p5 + eps)**(X5/tot2) *
            (p6 + eps)**(X6/tot2) *
            (p7 + eps)**(X7/tot2) *
            (p8 + eps)**(X8/tot2)
        )

        return -np.log(L) # Return the negative log-likelihood for minimization

    def table_counts(self, data):
        X1 = np.sum(data[:, 0] == 1)
        X2 = np.sum(data[:, 0] != data[:, 0])
        X3 = np.sum(data[:, 0] != data[:, 1])
        X4 = np.sum(data[:, 0] == 0)

        X5 = np.sum(data[:, 1] == 1)
        X6 = np.sum(data[:, 1] != data[:, 0])
        X7 = np.sum(data[:, 1] != data[:, 1])
        X8 = np.sum(data[:, 1] == 0)

        tot1 = X1 + X3 + X3 + X4
        tot2 =  X5 + X6 + X7 + X8
        return X1, X2, X3, X4, X5, X6, X7, X8, tot1, tot2

    def negative_log_likelihood(self, params, data):
        # Unpack the parameters
        alpha1, alpha2, beta1, beta2, theta1, theta2 = params
        p1, p2, p3, p4, p5, p6, p7, p8 = self.calculate_probabilities(alpha1, alpha2, beta1, beta2, theta1, theta2)

        X1, X2, X3, X4, X5, X6, X7, X8, tot1, tot2 = self.table_counts(data)

        eps = 1e-10
        L = (
            (p1 + eps)**(X1/tot1) *
            (p2 + eps)**(X2/tot1) *
            (p3 + eps)**(X3/tot1) *
            (p4 + eps)**(X4/tot1) *
            (p5 + eps)**(X5/tot2) *
            (p6 + eps)**(X6/tot2) *
            (p7 + eps)**(X7/tot2) *
            (p8 + eps)**(X8/tot2)
        )

        return -np.log(L) # Return the negative log-likelihood for minimization

    def mle(self, data):

        # Initial guess
        initial_guess = self.params #[0.5, 0.5, 0.5, 0.5, 0.5, 0.5]

        bounds = [(0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1)]

        # Perform minimization
        result = minimize(self.negative_log_likelihood, initial_guess, args=(data,), bounds=bounds)

        return result.x

    def table_counts(self, data):
        X1 = np.sum((data[:, 0] == 1) & (data[:, 1] == 1))  # Both classifiers predict 1
        X2 = np.sum((data[:, 0] == 1) & (data[:, 1] == 0))  # First predicts 1, second predicts 0
        X3 = np.sum((data[:, 0] == 0) & (data[:, 1] == 1))  # First predicts 0, second predicts 1
        X4 = np.sum((data[:, 0] == 0) & (data[:, 1] == 0))  # Both classifiers predict 0

        tot1 = X1 + X2 + X3 + X4  # Total counts
        return X1, X2, X3, X4, tot1

    def update(self, data):

         # Unpack the parameters
        alpha1, alpha2, beta1, beta2, theta1, theta2 = self.params

        a1, a2, b1, b2, t1, t2 = self.mle(data)

        # # Update parameters with a step proportional to the responsibilities and step_size
        # alpha1 += self.step_size * (alpha1 - a1)
        # alpha2 += self.step_size * (alpha2 - a2)
        # beta1  += self.step_size * (beta1 - b1)
        # beta2  += self.step_size * (beta2 - b2)
        # theta1 += self.step_size * (theta1 - t1)
        # theta2 += self.step_size * (theta2 - t2)
        #Update parameters based on comparison with initial values
        alpha1 += self.step_size * (a1 - alpha1) if a1 > alpha1 else -self.step_size * (alpha1 - a1)
        alpha2 += self.step_size * (a2 - alpha2) if a2 > alpha2 else -self.step_size * (alpha2 - a2)
        beta1 += self.step_size * (b1 - beta1) if b1 > beta1 else -self.step_size * (beta1 - b1)
        beta2 += self.step_size * (b2 - beta2) if b2 > beta2 else -self.step_size * (beta2 - b2)
        theta1 += self.step_size * (t1 - theta1) if t1 > theta1 else -self.step_size * (theta1 - t1)
        theta2 += self.step_size * (t2 - theta2) if t2 > theta2 else -self.step_size * (theta2 - t2)
        # alpha1 += self.step_size * a1 if a1 < alpha1 else -self.step_size * a1
        # alpha2 += self.step_size * a2 if a2 < alpha2 else -self.step_size * a2
        # beta1 += self.step_size * b1 if b1 < beta1 else -self.step_size * b1
        # beta2 += self.step_size * b2 if b2 < beta2 else -self.step_size * b2
        # theta1 += self.step_size * t1 if t1 < theta1 else -self.step_size * t1
        # theta2 += self.step_size * t2 if t2 < theta2 else -self.step_size * t2


        # Ensure parameters remain within valid range
        self.params = [
            max(0, min(1, alpha1)), max(0, min(1, alpha2)),
            max(0, min(1, beta1)), max(0, min(1, beta2)),
            max(0, min(1, theta1)), max(0, min(1, theta2))
        ]

    def _update(self, data):
        print(self.negative_log_likelihood(data))


def generate_data(n_samples, priors, se, sp):
    n_classes = len(priors)
    n_tests = len(se)

    data = np.zeros((n_samples, n_tests))
    true_classes = np.random.choice(n_classes, n_samples, p=priors)
    #print(true_classes)

    for i, c in enumerate(true_classes):
        for j in range(n_tests):
            if c == 1:
                data[i, j] = np.random.choice([0, 1], p=[1-se[j], se[j]])
            else:
                data[i, j] = np.random.choice([0, 1], p=[sp[j], 1-sp[j]])

    return data

# True parameters for generating synthetic data
true_priors = [0.6, 0.4]
true_se = [0.9, 0.7]
true_sp = [0.8, 0.6]

# Generate synthetic data
n_samples = 10000
data = generate_data(n_samples, true_priors, true_se, true_sp)
# Initialize the OnlineEMHuiWalter class
params = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
step_size = 0.01
em = OnlineEMHuiWalter(2, 2, params, step_size)
for i in range(n_samples):
    em.update(data[i].reshape(1, -1))
    print(em.params)

In [40]:
data

array([[1., 0.],
       [0., 1.],
       [1., 1.],
       ...,
       [1., 1.],
       [1., 1.],
       [1., 0.]])

In [47]:
from re import X
from scipy.optimize import minimize
import numpy as np


class OnlineEMHuiWalter:
    def __init__(self, num_classes, num_categories,
                 params, step_size=0.1):
        self.num_classes = num_classes
        self.num_categories = num_categories
        assert num_classes == 2
        assert num_categories == 2
        self.params = params
        self.step_size = step_size

    def expectation_step(self,  data):
        # Unpack the parameters
        alpha1, alpha2, beta1, beta2, theta1, theta2 = self.params

        # Unpack the data
        #X1, X2, X3, X4, X5, X6, X7, X8 = data

        X1 = np.sum(data[:, 0] == 1)
        X2 = np.sum(data[:, 0] != data[:, 0])
        X3 = np.sum(data[:, 0] != data[:, 1])
        X4 = np.sum(data[:, 0] == 0)

        X5 = np.sum(data[:, 1] == 1)
        X6 = np.sum(data[:, 1] != data[:, 0])
        X7 = np.sum(data[:, 1] != data[:, 1])
        X8 = np.sum(data[:, 1] == 0)

        # Calculate total counts for each population
        tot1 = X1 + X2 + X3 + X4
        tot2 = X5 + X6 + X7 + X8

        # Calculate probabilities for each cell in the first population
        p1 = theta1 * (1 - beta1) * (1 - beta2) + (1 - theta1) * alpha1 * alpha2
        p2 = theta1 * (1 - beta1) * beta2 + (1 - theta1) * alpha1 * (1 - alpha2)
        p3 = theta1 * beta1 * (1 - beta2) + (1 - theta1) * (1 - alpha1) * alpha2
        p4 = theta1 * beta1 * beta2 + (1 - theta1) * (1 - alpha1) * (1 - alpha2)

        # Calculate probabilities for each cell in the second population
        p5 = theta2 * (1 - beta1) * (1 - beta2) + (1 - theta2) * alpha1 * alpha2
        p6 = theta2 * (1 - beta1) * beta2 + (1 - theta2) * alpha1 * (1 - alpha2)
        p7 = theta2 * beta1 * (1 - beta2) + (1 - theta2) * (1 - alpha1) * alpha2
        p8 = theta2 * beta1 * beta2 + (1 - theta2) * (1 - alpha1) * (1 - alpha2)

        # Expected counts for each cell
        expected_counts = [
            tot1 * p1, tot1 * p2, tot1 * p3, tot1 * p4,
            tot2 * p5, tot2 * p6, tot2 * p7, tot2 * p8
        ]

        cell_probs = [
            p1, p2, p3, p4,
            p5, p6, p7, p8
        ]

        return expected_counts, cell_probs

    def calculate_probabilities(self, alpha1, alpha2, beta1, beta2, theta1, theta2):
        # Calculate probabilities for each cell in the first population
        p1 = theta1 * (1 - beta1) * (1 - beta2) + (1 - theta1) * alpha1 * alpha2
        p2 = theta1 * (1 - beta1) * beta2 + (1 - theta1) * alpha1 * (1 - alpha2)
        p3 = theta1 * beta1 * (1 - beta2) + (1 - theta1) * (1 - alpha1) * alpha2
        p4 = theta1 * beta1 * beta2 + (1 - theta1) * (1 - alpha1) * (1 - alpha2)

        # Calculate probabilities for each cell in the second population
        p5 = theta2 * (1 - beta1) * (1 - beta2) + (1 - theta2) * alpha1 * alpha2
        p6 = theta2 * (1 - beta1) * beta2 + (1 - theta2) * alpha1 * (1 - alpha2)
        p7 = theta2 * beta1 * (1 - beta2) + (1 - theta2) * (1 - alpha1) * alpha2
        p8 = theta2 * beta1 * beta2 + (1 - theta2) * (1 - alpha1) * (1 - alpha2)

        return p1, p2, p3, p4, p5, p6, p7, p8

    def negative_log_likelihood(self, data):
        # Unpack the parameters
        alpha1, alpha2, beta1, beta2, theta1, theta2 = self.params
        p1, p2, p3, p4, p5, p6, p7, p8 = self.calculate_probabilities(alpha1, alpha2, beta1, beta2, theta1, theta2)

        X1 = np.sum(data[:, 0] == 1)
        X2 = np.sum(data[:, 0] != data[:, 0])
        X3 = np.sum(data[:, 0] != data[:, 1])
        X4 = np.sum(data[:, 0] == 0)

        X5 = np.sum(data[:, 1] == 1)
        X6 = np.sum(data[:, 1] != data[:, 0])
        X7 = np.sum(data[:, 1] != data[:, 1])
        X8 = np.sum(data[:, 1] == 0)

        tot1 = X1 + X3 + X3 + X4
        tot2 =  X5 + X6 + X7 + X8

        eps = 1e-10
        L = (
            (p1 + eps)**(X1/tot1) *
            (p2 + eps)**(X2/tot1) *
            (p3 + eps)**(X3/tot1) *
            (p4 + eps)**(X4/tot1) *
            (p5 + eps)**(X5/tot2) *
            (p6 + eps)**(X6/tot2) *
            (p7 + eps)**(X7/tot2) *
            (p8 + eps)**(X8/tot2)
        )

        return -np.log(L) # Return the negative log-likelihood for minimization

    def table_counts(self, data):
        X1 = np.sum(data[:, 0] == 1)
        X2 = np.sum(data[:, 0] != data[:, 0])
        X3 = np.sum(data[:, 0] != data[:, 1])
        X4 = np.sum(data[:, 0] == 0)

        X5 = np.sum(data[:, 1] == 1)
        X6 = np.sum(data[:, 1] != data[:, 0])
        X7 = np.sum(data[:, 1] != data[:, 1])
        X8 = np.sum(data[:, 1] == 0)

        tot1 = X1 + X3 + X3 + X4
        tot2 =  X5 + X6 + X7 + X8
        return X1, X2, X3, X4, X5, X6, X7, X8, tot1, tot2

    def negative_log_likelihood(self, params, data):
        # Unpack the parameters
        alpha1, alpha2, beta1, beta2, theta1, theta2 = params
        p1, p2, p3, p4, p5, p6, p7, p8 = self.calculate_probabilities(alpha1, alpha2, beta1, beta2, theta1, theta2)

        X1, X2, X3, X4, X5, X6, X7, X8, tot1, tot2 = self.table_counts(data)

        eps = 1e-10
        L = (
            (p1 + eps)**(X1/tot1) *
            (p2 + eps)**(X2/tot1) *
            (p3 + eps)**(X3/tot1) *
            (p4 + eps)**(X4/tot1) *
            (p5 + eps)**(X5/tot2) *
            (p6 + eps)**(X6/tot2) *
            (p7 + eps)**(X7/tot2) *
            (p8 + eps)**(X8/tot2)
        )

        return -np.log(L) # Return the negative log-likelihood for minimization

    def mle(self, data):

        # Initial guess
        initial_guess = self.params #[0.5, 0.5, 0.5, 0.5, 0.5, 0.5]

        bounds = [(0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1)]

        # Perform minimization
        result = minimize(self.negative_log_likelihood, initial_guess, args=(data,), bounds=bounds)

        return result.x

    def table_counts(self, data):
        X1 = np.sum((data[:, 0] == 1) & (data[:, 1] == 1))  # Both classifiers predict 1
        X2 = np.sum((data[:, 0] == 1) & (data[:, 1] == 0))  # First predicts 1, second predicts 0
        X3 = np.sum((data[:, 0] == 0) & (data[:, 1] == 1))  # First predicts 0, second predicts 1
        X4 = np.sum((data[:, 0] == 0) & (data[:, 1] == 0))  # Both classifiers predict 0

        tot1 = X1 + X2 + X3 + X4  # Total counts
        return X1, X2, X3, X4, tot1

    def update(self, data):

         # Unpack the parameters
        alpha1, alpha2, beta1, beta2, theta1, theta2 = self.params

        a1, a2, b1, b2, t1, t2 = self.mle(data)

        # # Update parameters with a step proportional to the responsibilities and step_size
        # alpha1 += self.step_size * (alpha1 - a1)
        # alpha2 += self.step_size * (alpha2 - a2)
        # beta1  += self.step_size * (beta1 - b1)
        # beta2  += self.step_size * (beta2 - b2)
        # theta1 += self.step_size * (theta1 - t1)
        # theta2 += self.step_size * (theta2 - t2)
        #Update parameters based on comparison with initial values
        alpha1 += self.step_size * (a1 - alpha1) if a1 > alpha1 else -self.step_size * (alpha1 - a1)
        alpha2 += self.step_size * (a2 - alpha2) if a2 > alpha2 else -self.step_size * (alpha2 - a2)
        beta1 += self.step_size * (b1 - beta1) if b1 > beta1 else -self.step_size * (beta1 - b1)
        beta2 += self.step_size * (b2 - beta2) if b2 > beta2 else -self.step_size * (beta2 - b2)
        theta1 += self.step_size * (t1 - theta1) if t1 > theta1 else -self.step_size * (theta1 - t1)
        theta2 += self.step_size * (t2 - theta2) if t2 > theta2 else -self.step_size * (theta2 - t2)
        # alpha1 += self.step_size * a1 if a1 < alpha1 else -self.step_size * a1
        # alpha2 += self.step_size * a2 if a2 < alpha2 else -self.step_size * a2
        # beta1 += self.step_size * b1 if b1 < beta1 else -self.step_size * b1
        # beta2 += self.step_size * b2 if b2 < beta2 else -self.step_size * b2
        # theta1 += self.step_size * t1 if t1 < theta1 else -self.step_size * t1
        # theta2 += self.step_size * t2 if t2 < theta2 else -self.step_size * t2


        # Ensure parameters remain within valid range
        self.params = [
            max(0, min(1, alpha1)), max(0, min(1, alpha2)),
            max(0, min(1, beta1)), max(0, min(1, beta2)),
            max(0, min(1, theta1)), max(0, min(1, theta2))
        ]

    def _update(self, data):
        print(self.negative_log_likelihood(data))


def generate_data(n_samples, n_classes, prior, se, sp):
    #n_classes = len(priors)
    n_tests = len(se)

    data = np.zeros((n_samples, n_tests))
    true_classes = np.random.choice(n_classes, n_samples, p=priors)
    #print(true_classes)

    for i, c in enumerate(true_classes):
        for j in range(n_tests):
            if c == 1:
                data[i, j] = np.random.choice([0, 1], p=[1-se[j], se[j]])
            else:
                data[i, j] = np.random.choice([0, 1], p=[sp[j], 1-sp[j]])

    return data

# True parameters for generating synthetic data
true_priors = [0.6, 0.4]
true_se = [0.9, 0.7]
true_sp = [0.8, 0.6]
n_classes = 2

# Generate synthetic data
n_samples = 10000
pop_one_data = generate_data(n_samples, n_classes, true_priors[0], true_se, true_sp)
pop_two_data = generate_data(n_samples, n_classes, true_priors[1], true_se, true_sp)
# Initialize the OnlineEMHuiWalter class
params = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
step_size = 0.01
em = OnlineEMHuiWalter(2, 2, params, step_size)
for i in range(n_samples):
    #em.update(data[i].reshape(1, -1))
    print(pop_one_data[i], pop_two_data[i])
    #print(em.params)

Streaming output truncated to the last 5000 lines.
[1. 1.] [0. 0.]
[1. 0.] [1. 1.]
[1. 1.] [0. 0.]
[0. 1.] [1. 1.]
[0. 1.] [0. 0.]
[0. 0.] [1. 0.]
[1. 0.] [1. 1.]
[1. 1.] [0. 0.]
[0. 1.] [0. 0.]
[0. 0.] [1. 1.]
[0. 1.] [1. 1.]
[1. 0.] [0. 0.]
[0. 0.] [1. 1.]
[1. 1.] [1. 1.]
[1. 0.] [1. 1.]
[0. 1.] [1. 1.]
[0. 1.] [0. 0.]
[1. 1.] [1. 1.]
[0. 1.] [1. 1.]
[1. 0.] [1. 1.]
[1. 1.] [1. 1.]
[0. 0.] [1. 1.]
[0. 0.] [0. 1.]
[0. 0.] [1. 0.]
[1. 1.] [0. 0.]
[1. 1.] [1. 0.]
[0. 1.] [1. 1.]
[0. 0.] [0. 1.]
[0. 0.] [0. 0.]
[1. 0.] [1. 1.]
[0. 0.] [1. 1.]
[1. 0.] [1. 0.]
[1. 1.] [1. 0.]
[0. 0.] [1. 0.]
[1. 0.] [1. 1.]
[0. 0.] [0. 0.]
[0. 1.] [1. 0.]
[0. 1.] [1. 1.]
[0. 0.] [0. 1.]
[1. 0.] [0. 0.]
[1. 1.] [0. 1.]
[0. 0.] [1. 0.]
[0. 0.] [1. 1.]
[1. 0.] [0. 0.]
[0. 0.] [1. 1.]
[1. 0.] [1. 1.]
[1. 1.] [1. 1.]
[0. 0.] [0. 0.]
[1. 1.] [1. 0.]
[0. 1.] [0. 0.]
[1. 1.] [0. 1.]
[0. 1.] [1. 1.]
[0. 1.] [1. 1.]
[1. 1.] [0. 0.]
[0. 0.] [1. 1.]
[1. 1.] [1. 1.]
[1. 1.] [0. 1.]
[0. 0.] [1. 1.]
[0. 0.] [1. 1.]
[0. 1

In [49]:
pop_one_data

array([[0., 1.],
       [1., 1.],
       [0., 1.],
       ...,
       [1., 1.],
       [1., 0.],
       [0., 0.]])

In [51]:
em.table_counts(pop_two_data)

(3487, 2056, 1873, 2584, 10000)

In [4]:
from scipy.optimize import minimize
import numpy as np


class OnlineEMHuiWalter:
    def __init__(self, params, step_size=0.1):
        self.params = params
        self.step_size = step_size
        self.alpha1 = []
        self.alpha2 = []
        self.beta1 = []
        self.beta2 = []
        self.theta1 = []
        self.theta2 = []


    def calculate_probabilities(self, alpha1, alpha2, beta1, beta2, theta1, theta2):
        # Calculate probabilities for each cell in the first population
        p1 = theta1 * (1 - beta1) * (1 - beta2) + (1 - theta1) * alpha1 * alpha2
        p2 = theta1 * (1 - beta1) * beta2 + (1 - theta1) * alpha1 * (1 - alpha2)
        p3 = theta1 * beta1 * (1 - beta2) + (1 - theta1) * (1 - alpha1) * alpha2
        p4 = theta1 * beta1 * beta2 + (1 - theta1) * (1 - alpha1) * (1 - alpha2)

        # Calculate probabilities for each cell in the second population
        p5 = theta2 * (1 - beta1) * (1 - beta2) + (1 - theta2) * alpha1 * alpha2
        p6 = theta2 * (1 - beta1) * beta2 + (1 - theta2) * alpha1 * (1 - alpha2)
        p7 = theta2 * beta1 * (1 - beta2) + (1 - theta2) * (1 - alpha1) * alpha2
        p8 = theta2 * beta1 * beta2 + (1 - theta2) * (1 - alpha1) * (1 - alpha2)

        return p1, p2, p3, p4, p5, p6, p7, p8

    def negative_log_likelihood(self, params, pop_one_data, pop_two_data):
        # Unpack the parameters
        alpha1, alpha2, beta1, beta2, theta1, theta2 = params
        p1, p2, p3, p4, p5, p6, p7, p8 = self.calculate_probabilities(alpha1, alpha2, beta1, beta2, theta1, theta2)

        X1, X2, X3, X4, tot1 = self.table_counts(pop_one_data)
        X5, X6, X7, X8, tot2 = self.table_counts(pop_two_data)

        eps = 1e-10
        L = (
            (p1 + eps)**(X1/tot1) *
            (p2 + eps)**(X2/tot1) *
            (p3 + eps)**(X3/tot1) *
            (p4 + eps)**(X4/tot1) *
            (p5 + eps)**(X5/tot2) *
            (p6 + eps)**(X6/tot2) *
            (p7 + eps)**(X7/tot2) *
            (p8 + eps)**(X8/tot2)
        )

        return -np.log(L) # Return the negative log-likelihood for minimization

    def mle(self, pop_one_data, pop_two_data):

        # Initial guess
        initial_guess = self.params #[0.5, 0.5, 0.5, 0.5, 0.5, 0.5]

        bounds = [(0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1)]

        # Perform minimization
        result = minimize(self.negative_log_likelihood, initial_guess, args=(pop_one_data, pop_two_data,), bounds=bounds)

        return result.x

    def table_counts(self, data):
        X1 = np.sum((data[:, 0] == 1) & (data[:, 1] == 1))  # Both classifiers predict 1
        X2 = np.sum((data[:, 0] == 1) & (data[:, 1] == 0))  # First predicts 1, second predicts 0
        X3 = np.sum((data[:, 0] == 0) & (data[:, 1] == 1))  # First predicts 0, second predicts 1
        X4 = np.sum((data[:, 0] == 0) & (data[:, 1] == 0))  # Both classifiers predict 0

        tot1 = X1 + X2 + X3 + X4  # Total counts
        return X1, X2, X3, X4, tot1

    def update(self, pop_one_data, pop_two_data):

         # Unpack the parameters
        alpha1, alpha2, beta1, beta2, theta1, theta2 = self.params

        a1, a2, b1, b2, t1, t2 = self.mle(pop_one_data, pop_two_data)

        self.alpha1.append(a1)
        self.alpha2.append(a2)
        self.beta1.append(b1)
        self.beta2.append(b2)
        self.theta1.append(t1)
        self.theta2.append(t2)

        alpha1 =  np.mean(self.alpha1)
        alpha2 = np.mean(self.alpha2)
        beta1  = np.mean(self.beta1)
        beta2  = np.mean(self.beta2)
        theta1 = np.mean(self.theta1)
        theta2 = np.mean(self.theta2)

        # Ensure parameters remain within valid range
        self.params = [
            max(0, min(1, alpha1)), max(0, min(1, alpha2)),
            max(0, min(1, beta1)), max(0, min(1, beta2)),
            max(0, min(1, theta1)), max(0, min(1, theta2))
        ]


def generate_data(n_samples, n_classes, prior, se, sp):
    n_tests = len(se)

    data = np.zeros((n_samples, n_tests))
    true_classes = np.random.choice(n_classes, n_samples, p=[prior, 1 - prior])

    for i, c in enumerate(true_classes):
        for j in range(n_tests):
            if c == 1:
                data[i, j] = np.random.choice([0, 1], p=[1-se[j], se[j]])
            else:
                data[i, j] = np.random.choice([0, 1], p=[sp[j], 1-sp[j]])

    return data

# True parameters for generating synthetic data
true_priors = [0.6, 0.4]
true_se = [0.9, 0.7]
true_sp = [0.8, 0.6]
n_classes = 2

# Generate synthetic data
n_samples = 100000
pop_one_data = generate_data(n_samples, n_classes, true_priors[0], true_se, true_sp)
pop_two_data = generate_data(n_samples, n_classes, true_priors[1], true_se, true_sp)
# Initialize the OnlineEMHuiWalter class
init_params = [0.6, 0.4, 0.9, 0.7, 0.8, 0.6]
step_size = 0.001
em = OnlineEMHuiWalter(init_params, step_size)
for i in range(n_samples):
    #print(pop_one_data[i])
    em.update(pop_one_data[:i], pop_two_data[:i])
    #print(pop_one_data[i], pop_two_data[i])

print(em.params)

<ipython-input-4-5312d22296a0>:58: RuntimeWarning: invalid value encountered in scalar divide
  (p1 + eps)**(X1/tot1) *
<ipython-input-4-5312d22296a0>:59: RuntimeWarning: invalid value encountered in scalar divide
  (p2 + eps)**(X2/tot1) *
<ipython-input-4-5312d22296a0>:60: RuntimeWarning: invalid value encountered in scalar divide
  (p3 + eps)**(X3/tot1) *
<ipython-input-4-5312d22296a0>:61: RuntimeWarning: invalid value encountered in scalar divide
  (p4 + eps)**(X4/tot1) *
<ipython-input-4-5312d22296a0>:62: RuntimeWarning: invalid value encountered in scalar divide
  (p5 + eps)**(X5/tot2) *
<ipython-input-4-5312d22296a0>:63: RuntimeWarning: invalid value encountered in scalar divide
  (p6 + eps)**(X6/tot2) *
<ipython-input-4-5312d22296a0>:64: RuntimeWarning: invalid value encountered in scalar divide
  (p7 + eps)**(X7/tot2) *
<ipython-input-4-5312d22296a0>:65: RuntimeWarning: invalid value encountered in scalar divide
  (p8 + eps)**(X8/tot2)


[0.8847202675953674, 0.7051717236978017, 0.790840753724, 0.6106034842924211, 0.598741790198845, 0.3948955007080811]


In [60]:
pop_one_data[:i], i, pop_one_data[:i].shape

(array([[0., 1.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 1.],
        [1., 0.],
        [1., 0.]]),
 534,
 (534, 2))